# Import data

In [ ]:
# importing all necessary libraries
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from tqdm.notebook import tqdm as tqdm

import plotly.graph_objs as go #visualization library
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf #autocorrelation test
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller #stationarity test
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from datetime import datetime, timedelta
import statsmodels.api as sm
import gc
from pylab import rcParams
import random
import os
import time
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import gc
import lightgbm as lgb
import time
import numpy as np
import pandas as pd
import os
import time
import matplotlib.dates as mdates
import gc
import lightgbm as lgb
import time

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import multiprocessing as mp
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
# Reading dataset's ...
calender    = pd.read_csv('https://media.githubusercontent.com/media/minhhieu9800/Python-MSE/main/Data/calendar.csv')
train_sales = pd.read_csv('https://media.githubusercontent.com/media/minhhieu9800/Python-MSE/main/Data/sales_train_evaluation.csv')
df_val      = pd.read_csv('https://media.githubusercontent.com/media/minhhieu9800/Python-MSE/main/Data/sales_train_validation.csv')
sell_prices = pd.read_csv('https://media.githubusercontent.com/media/minhhieu9800/Python-MSE/main/Data/sell_prices.csv')

In [ ]:
train_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [ ]:
holiday = ['NewYear', 'OrthodoxChristmas', 'MartinLutherKingDay', 'SuperBowl', 'PresidentsDay', 'StPatricksDay', 'Easter', 'Cinco De Mayo', 'IndependenceDay', 'EidAlAdha', 'Thanksgiving', 'Christmas']
weekend = ['Saturday', 'Sunday']

def is_holiday(x):
    if x in holiday:
        return 1
    else:
        return 0

def is_weekend(x):
    if x in weekend:
        return 1
    else:
        return 0

In [ ]:
calender['is_holiday_1'] = calender['event_name_1'].apply(is_holiday)
calender['is_holiday_2'] = calender['event_name_2'].apply(is_holiday)
calender['is_holiday'] = calender[['is_holiday_1','is_holiday_2']].max(axis=1)
calender['is_weekend'] = calender['weekday'].apply(is_weekend)

In [ ]:
calender = calender.drop(['weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis='columns')

In [ ]:
del_col = []
for x in range(1851):
    del_col.append('d_' + str(x+1))
train_sales =train_sales.drop(del_col, axis='columns')

In [ ]:
train_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1852,d_1853,d_1854,d_1855,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,1,0,1,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,1,2,3,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,3,1,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,0,1,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,1,0,0,0,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [ ]:
train_sales = train_sales.melt(['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='qty')
print(train_sales.shape)
train_sales.head()

(2744100, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,d,qty
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,1
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0


In [ ]:
train_sales = pd.merge(train_sales, calender, how='left', on='d')
train_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,qty,date,wm_yr_wk,snap_CA,snap_TX,snap_WI,is_holiday_1,is_holiday_2,is_holiday,is_weekend
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,1,2016-02-23,11604,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0


In [ ]:
train_sales = pd.merge(train_sales, sell_prices, how='left', on=['item_id', 'wm_yr_wk', 'store_id'])
train_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,qty,date,wm_yr_wk,snap_CA,snap_TX,snap_WI,is_holiday_1,is_holiday_2,is_holiday,is_weekend,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0,8.26
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0,3.97
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0,2.97
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,1,2016-02-23,11604,0,0,0,0,0,0,0,4.64
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0,2.88


In [ ]:
train_sales_test = train_sales.query('d == "d_1852"')
train_sales_test = train_sales_test[['id', 'store_id', 'item_id', 'dept_id', 'cat_id', 'state_id', 'd', 'qty', 'sell_price']]
train_sales_test['qty'] =train_sales_test['d'].apply(lambda x: int(x.replace(x, '0')))

In [ ]:
train_sales_test

,id,store_id,item_id,dept_id,cat_id,state_id,d,qty,sell_price
0,HOBBIES_1_001_CA_1_evaluation,CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA,d_1852,0,8.26
1,HOBBIES_1_002_CA_1_evaluation,CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA,d_1852,0,3.97
2,HOBBIES_1_003_CA_1_evaluation,CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA,d_1852,0,2.97
3,HOBBIES_1_004_CA_1_evaluation,CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA,d_1852,0,4.64
4,HOBBIES_1_005_CA_1_evaluation,CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA,d_1852,0,2.88
...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,WI_3,FOODS_3_823,FOODS_3,FOODS,WI,d_1852,0,2.88
30486,FOODS_3_824_WI_3_evaluation,WI_3,FOODS_3_824,FOODS_3,FOODS,WI,d_1852,0,2.00
30487,FOODS_3_825_WI_3_evaluation,WI_3,FOODS_3_825,FOODS_3,FOODS,WI,d_1852,0,3.98
30488,FOODS_3_826_WI_3_evaluation,WI_3,FOODS_3_826,FOODS_3,FOODS,WI,d_1852,0,1.28


In [ ]:
tmp_df =train_sales_test
for x in range(28):
    train_sales_test =train_sales_test.append(tmp_df)

In [ ]:
train_sales_test

,id,store_id,item_id,dept_id,cat_id,state_id,d,qty,sell_price
0,HOBBIES_1_001_CA_1_evaluation,CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA,d_1852,0,8.26
1,HOBBIES_1_002_CA_1_evaluation,CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA,d_1852,0,3.97
2,HOBBIES_1_003_CA_1_evaluation,CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA,d_1852,0,2.97
3,HOBBIES_1_004_CA_1_evaluation,CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA,d_1852,0,4.64
4,HOBBIES_1_005_CA_1_evaluation,CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA,d_1852,0,2.88
...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,WI_3,FOODS_3_823,FOODS_3,FOODS,WI,d_1852,0,2.88
30486,FOODS_3_824_WI_3_evaluation,WI_3,FOODS_3_824,FOODS_3,FOODS,WI,d_1852,0,2.00
30487,FOODS_3_825_WI_3_evaluation,WI_3,FOODS_3_825,FOODS_3,FOODS,WI,d_1852,0,3.98
30488,FOODS_3_826_WI_3_evaluation,WI_3,FOODS_3_826,FOODS_3,FOODS,WI,d_1852,0,1.28


In [ ]:
train_sales_test =train_sales_test.reset_index(drop=True)

In [ ]:
lst_d = []
i = 0
lst_index = train_sales_test.index
for x in lst_index:
    lst_d.append('d_' + str(((lst_index[i]) // 30490) + 1942))
    i = i + 1

In [ ]:
train_sales_test['d'] = lst_d

In [ ]:
train_sales_test = pd.merge(train_sales_test,calender, how='left', on='d')
train_sales_test = pd.merge(train_sales_test, sell_prices ,how='left', on=['item_id', 'wm_yr_wk', 'store_id'])

In [ ]:
train_sales_test.head()

,id,store_id,item_id,dept_id,cat_id,state_id,d,qty,sell_price_x,date,wm_yr_wk,snap_CA,snap_TX,snap_WI,is_holiday_1,is_holiday_2,is_holiday,is_weekend,sell_price_y
0,HOBBIES_1_001_CA_1_evaluation,CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA,d_1942,0,8.26,2016-05-23,11617.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.38
1,HOBBIES_1_002_CA_1_evaluation,CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA,d_1942,0,3.97,2016-05-23,11617.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.97
2,HOBBIES_1_003_CA_1_evaluation,CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA,d_1942,0,2.97,2016-05-23,11617.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.97
3,HOBBIES_1_004_CA_1_evaluation,CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA,d_1942,0,4.64,2016-05-23,11617.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.64
4,HOBBIES_1_005_CA_1_evaluation,CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA,d_1942,0,2.88,2016-05-23,11617.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.88


In [ ]:
train_sales = pd.get_dummies(data=train_sales, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])

In [ ]:
train_sales_test = pd.get_dummies(data=train_sales_test, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])

In [ ]:
train_sales = train_sales.drop(['snap_CA', 'snap_TX', 'snap_WI'], axis='columns')  

In [ ]:
train_sales_test = train_sales_test.drop(['sell_price_x', 'snap_CA', 'snap_TX', 'snap_WI'], axis='columns')
train_sales_test = train_sales_test.rename(columns={'sell_price_y': 'sell_price'})

# Model Training

In [ ]:
# Data hiện tại (từng ID sản phẩm trong 23/2/2016 - 22/5/2016)
# Data để train
train_sales

,id,item_id,d,qty,date,wm_yr_wk,is_holiday_1,is_holiday_2,is_holiday,is_weekend,...,store_id_CA_4,store_id_TX_1,store_id_TX_2,store_id_TX_3,store_id_WI_1,store_id_WI_2,store_id_WI_3,state_id_CA,state_id_TX,state_id_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,d_1852,0,2016-02-23,11604,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,d_1852,0,2016-02-23,11604,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,d_1852,0,2016-02-23,11604,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,d_1852,1,2016-02-23,11604,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,d_1852,0,2016-02-23,11604,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2744095,FOODS_3_823_WI_3_evaluation,FOODS_3_823,d_1941,1,2016-05-22,11617,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
2744096,FOODS_3_824_WI_3_evaluation,FOODS_3_824,d_1941,0,2016-05-22,11617,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
2744097,FOODS_3_825_WI_3_evaluation,FOODS_3_825,d_1941,2,2016-05-22,11617,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
2744098,FOODS_3_826_WI_3_evaluation,FOODS_3_826,d_1941,0,2016-05-22,11617,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1


In [ ]:
# Data để test prediction
train_sales_test

,id,item_id,d,qty,date,wm_yr_wk,is_holiday_1,is_holiday_2,is_holiday,is_weekend,...,store_id_CA_4,store_id_TX_1,store_id_TX_2,store_id_TX_3,store_id_WI_1,store_id_WI_2,store_id_WI_3,state_id_CA,state_id_TX,state_id_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,d_1942,0,2016-05-23,11617.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,d_1942,0,2016-05-23,11617.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,d_1942,0,2016-05-23,11617.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,d_1942,0,2016-05-23,11617.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,d_1942,0,2016-05-23,11617.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884205,FOODS_3_823_WI_3_evaluation,FOODS_3_823,d_1970,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,1
884206,FOODS_3_824_WI_3_evaluation,FOODS_3_824,d_1970,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,1
884207,FOODS_3_825_WI_3_evaluation,FOODS_3_825,d_1970,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,1
884208,FOODS_3_826_WI_3_evaluation,FOODS_3_826,d_1970,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,1


In [ ]:
from sklearn.model_selection import train_test_split

# Dự đoán số lượng sản phẩm bán ra
target_col = 'qty'

# Loại bỏ các cột thừa
exclude_cols = ['id', 'item_id', 'd', 'date', 'wm_yr_wk']

# Giữ lại các cột khác
feature_cols = [col for col in train_sales.columns if col not in exclude_cols]

# Tạo numpy array
y = np.array(train_sales[target_col])
X = np.array(train_sales[feature_cols])

# Tách array ngẫu nhiên
X_train, X_test, y_train, y_test = \
 train_test_split(X, y, test_size=0.3, random_state=1234)

In [ ]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)

[[5. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [3. 0. 0. ... 0. 1. 0.]
 [2. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[[2. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 [3. 0. 0. ... 0. 0. 1.]]
[5 1 0 ... 3 2 0]
[2 0 0 ... 0 0 3]


## Mô hình LightGBM

In [ ]:
import lightgbm as lgb

# Tạo dữ liệu cho model
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test)

# Cài đặt
params = {
    'boosting_type': 'gbdt',    # áp dụng Cây Quyết định (Gradient Boosting Decision Tree)
    'metric': 'rmse',           # thước đo hiệu quả: Lỗi trung bình bình phương gốc
    'objective': 'regression',  # thuật toán Hồi qui
    'n_jobs': -1,               # vô hạn thread
    'seed': 236,
    'learning_rate': 0.01,      # tốc độ học
    'bagging_fraction': 0.75,   # thiết lập bagging
    'bagging_freq': 10, 
    'colsample_bytree': 0.75}

# Chạy machine learning, training dựa vào data chính
model = lgb.train(params, lgb_train, num_boost_round=2500, early_stopping_rounds=50, valid_sets = [lgb_train, lgb_eval], verbose_eval=100)

Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 1.68222	valid_1's rmse: 1.66387
[200]	training's rmse: 0.79235	valid_1's rmse: 0.776693
[300]	training's rmse: 0.409673	valid_1's rmse: 0.391969
[400]	training's rmse: 0.252703	valid_1's rmse: 0.230078
[500]	training's rmse: 0.180503	valid_1's rmse: 0.153709
[600]	training's rmse: 0.151524	valid_1's rmse: 0.121863
[700]	training's rmse: 0.137441	valid_1's rmse: 0.106603
[800]	training's rmse: 0.129903	valid_1's rmse: 0.0982292
[900]	training's rmse: 0.125156	valid_1's rmse: 0.0931496
[1000]	training's rmse: 0.122392	valid_1's rmse: 0.0897334
[1100]	training's rmse: 0.120223	valid_1's rmse: 0.0868855
[1200]	training's rmse: 0.118617	valid_1's rmse: 0.0853847
[1300]	training's rmse: 0.117583	valid_1's rmse: 0.0838488
[1400]	training's rmse: 0.116705	valid_1's rmse: 0.083269
Early stopping, best iteration is:
[1381]	training's rmse: 0.116868	valid_1's rmse: 0.082419


=> Training loss là 0.1168 còn validation loss là 0.082

In [ ]:
# Tạo prediction dựa trên mẫu data test
pred = model.predict(train_sales_test[feature_cols])

In [ ]:
len(pred)

884210

In [ ]:
train_sales_test['pred_qty'] = pred

In [ ]:
train_sales_test

,id,item_id,d,qty,date,wm_yr_wk,is_holiday_1,is_holiday_2,is_holiday,is_weekend,...,store_id_TX_1,store_id_TX_2,store_id_TX_3,store_id_WI_1,store_id_WI_2,store_id_WI_3,state_id_CA,state_id_TX,state_id_WI,pred_qty
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,d_1942,0,2016-05-23,11617.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0.010989
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,d_1942,0,2016-05-23,11617.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0.011306
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,d_1942,0,2016-05-23,11617.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0.010672
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,d_1942,0,2016-05-23,11617.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0.010241
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,d_1942,0,2016-05-23,11617.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0.012270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884205,FOODS_3_823_WI_3_evaluation,FOODS_3_823,d_1970,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,1,0.087166
884206,FOODS_3_824_WI_3_evaluation,FOODS_3_824,d_1970,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,1,0.087166
884207,FOODS_3_825_WI_3_evaluation,FOODS_3_825,d_1970,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,1,0.087166
884208,FOODS_3_826_WI_3_evaluation,FOODS_3_826,d_1970,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,1,0.087166


In [ ]:
# Tổng hợp ra dự đoán
predictions = train_sales_test[['id', 'date', 'pred_qty']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'pred_qty').reset_index()

# Bỏ cột thừa
predictions = predictions.drop(predictions.columns[1], axis=1)

# Đổi tên cột ngày thành F{ngày}
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

predictions

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_evaluation,-0.007053,-0.007053,-0.007053,-0.007053,-0.007053,-0.005225,-0.005225,-0.007053,-0.007053,...,-0.007053,-0.005225,-0.005225,-0.007053,-0.007053,-0.007053,-0.007053,-0.007053,-0.005225,-0.005225
1,FOODS_1_001_CA_2_evaluation,0.015451,0.015451,0.015451,0.015451,0.015451,0.018480,0.018480,0.015451,0.015451,...,0.015451,0.018480,0.018480,0.015451,0.015451,0.015451,0.015451,0.015451,0.018480,0.018480
2,FOODS_1_001_CA_3_evaluation,0.001406,0.001406,0.001406,0.001406,0.001406,0.005404,0.005404,0.001406,0.001406,...,0.001406,0.005404,0.005404,0.001406,0.001406,0.001406,0.001406,0.001406,0.005404,0.005404
3,FOODS_1_001_CA_4_evaluation,-0.005411,-0.005411,-0.005411,-0.005411,-0.005411,-0.006876,-0.006876,-0.005411,-0.005411,...,-0.005411,-0.006876,-0.006876,-0.005411,-0.005411,-0.005411,-0.005411,-0.005411,-0.006876,-0.006876
4,FOODS_1_001_TX_1_evaluation,-0.008828,-0.008828,-0.008828,-0.008828,-0.008828,-0.014365,-0.014365,-0.008828,-0.008828,...,-0.008828,-0.014365,-0.014365,-0.008828,-0.008828,-0.008828,-0.008828,-0.008828,-0.014365,-0.014365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,-0.000643,-0.000643,-0.000643,-0.000643,-0.000643,-0.001027,-0.001027,-0.000643,-0.000643,...,-0.000643,-0.001027,-0.001027,-0.000643,-0.000643,-0.000643,-0.000643,-0.000643,-0.001027,-0.001027
30486,HOUSEHOLD_2_516_TX_3_evaluation,-0.000649,-0.000649,-0.000649,-0.000649,-0.000649,-0.001033,-0.001033,-0.000649,-0.000649,...,-0.000649,-0.001033,-0.001033,-0.000649,-0.000649,-0.000649,-0.000649,-0.000649,-0.001033,-0.001033
30487,HOUSEHOLD_2_516_WI_1_evaluation,-0.000834,-0.000834,-0.000834,-0.000834,-0.000834,-0.002167,-0.002167,-0.000834,-0.000834,...,-0.000834,-0.002167,-0.002167,-0.000834,-0.000834,-0.000834,-0.000834,-0.000834,-0.002167,-0.002167
30488,HOUSEHOLD_2_516_WI_2_evaluation,-0.005178,-0.005178,-0.005178,-0.005178,-0.005178,-0.004168,-0.004168,-0.005178,-0.005178,...,-0.005178,-0.004168,-0.004168,-0.005178,-0.005178,-0.005178,-0.005178,-0.005178,-0.004168,-0.004168


In [ ]:
# So sánh giá trị dự đoán với giá trị thật
x = 2744099 + 1 - 853720
df_val = train_sales[x:]

predictions_v = df_val[['id', 'date', 'qty']]
predictions_v = pd.pivot(predictions_v, index = 'id', columns = 'date', values = 'qty').reset_index()
predictions_v

date,id,2016-04-25,2016-04-26,2016-04-27,2016-04-28,2016-04-29,2016-04-30,2016-05-01,2016-05-02,2016-05-03,...,2016-05-13,2016-05-14,2016-05-15,2016-05-16,2016-05-17,2016-05-18,2016-05-19,2016-05-20,2016-05-21,2016-05-22
0,FOODS_1_001_CA_1_evaluation,2,0,0,0,0,1,1,0,6,...,2,3,1,0,0,0,1,0,0,0
1,FOODS_1_001_CA_2_evaluation,0,3,0,0,0,1,4,0,0,...,1,0,0,1,1,0,0,1,2,0
2,FOODS_1_001_CA_3_evaluation,1,0,1,0,8,1,0,0,1,...,1,2,2,0,0,1,0,3,2,2
3,FOODS_1_001_CA_4_evaluation,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,0
4,FOODS_1_001_TX_1_evaluation,0,0,1,0,1,0,1,1,1,...,1,1,1,1,5,0,2,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
30486,HOUSEHOLD_2_516_TX_3_evaluation,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,1,2
30487,HOUSEHOLD_2_516_WI_1_evaluation,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
30488,HOUSEHOLD_2_516_WI_2_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [ ]:
predictions_v['id'] = predictions['id'].apply(lambda x: x.replace('evaluation', 'validation'))
predictions_v.head()

date,id,2016-04-25,2016-04-26,2016-04-27,2016-04-28,2016-04-29,2016-04-30,2016-05-01,2016-05-02,2016-05-03,...,2016-05-13,2016-05-14,2016-05-15,2016-05-16,2016-05-17,2016-05-18,2016-05-19,2016-05-20,2016-05-21,2016-05-22
0,FOODS_1_001_CA_1_validation,2,0,0,0,0,1,1,0,6,...,2,3,1,0,0,0,1,0,0,0
1,FOODS_1_001_CA_2_validation,0,3,0,0,0,1,4,0,0,...,1,0,0,1,1,0,0,1,2,0
2,FOODS_1_001_CA_3_validation,1,0,1,0,8,1,0,0,1,...,1,2,2,0,0,1,0,3,2,2
3,FOODS_1_001_CA_4_validation,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,0
4,FOODS_1_001_TX_1_validation,0,0,1,0,1,0,1,1,1,...,1,1,1,1,5,0,2,2,0,2


In [ ]:
predictions_v.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
predictions_v.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,2,0,0,0,0,1,1,0,6,...,2,3,1,0,0,0,1,0,0,0
1,FOODS_1_001_CA_2_validation,0,3,0,0,0,1,4,0,0,...,1,0,0,1,1,0,0,1,2,0
2,FOODS_1_001_CA_3_validation,1,0,1,0,8,1,0,0,1,...,1,2,2,0,0,1,0,3,2,2
3,FOODS_1_001_CA_4_validation,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,0
4,FOODS_1_001_TX_1_validation,0,0,1,0,1,0,1,1,1,...,1,1,1,1,5,0,2,2,0,2


## Mô hình LSTM

In [ ]:
# Sử dụng scaling với biên độ 0-1
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))

# Drop các cột dư thừa và apply scaling
dt_scaled = sc.fit_transform(train_sales.drop(['id', 'item_id', 'd', 'date', 'wm_yr_wk'],axis=1))

In [ ]:
timesteps = 14
startDay = 350
X_train = []
y_train = []
for i in range(timesteps, 1941 - startDay):
    X_train.append(dt_scaled[i-timesteps:i])
    y_train.append(dt_scaled[i][0:30490]) 

In [ ]:
dt_scaled[0][0:30490]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.24383724, 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        ])

In [ ]:
# Convert sang data để feed cho model
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
# Importing package của Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Khởi tạo mạch thần kinh hồi qui
regressor = Sequential()

# Gắn lớp LSTM đầu và chính quy hoá Dropout
layer_1_units=50
regressor.add(LSTM(units = layer_1_units, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))

# Gắn lớp LSTM thứ ba và chính quy hoá Dropout
layer_3_units=400
regressor.add(LSTM(units = layer_3_units, return_sequences = True))
regressor.add(Dropout(0.2))

# Gắn lớp LSTM thứ ba và chính quy hoá Dropout
layer_3_units=400
regressor.add(LSTM(units = layer_3_units))
regressor.add(Dropout(0.2))

# Gắn lớp output
regressor.add(Dense(units = 29))

# Chạy mạch thần kinh hồi qui
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Áp dụng RNN vào data train
epoch_no=32
batch_size_RNN=44
regressor.fit(X_train, y_train, epochs = epoch_no, batch_size = batch_size_RNN)

Epoch 1/32
36/36 [==============================] - 18s 308ms/step - loss: 0.0173
Epoch 2/32
36/36 [==============================] - 12s 327ms/step - loss: 0.0043
Epoch 3/32
36/36 [==============================] - 11s 309ms/step - loss: 0.0037
Epoch 4/32
36/36 [==============================] - 12s 328ms/step - loss: 0.0035
Epoch 5/32
36/36 [==============================] - 11s 310ms/step - loss: 0.0034
Epoch 6/32
36/36 [==============================] - 11s 317ms/step - loss: 0.0031
Epoch 7/32
36/36 [==============================] - 12s 338ms/step - loss: 0.0028
Epoch 8/32
36/36 [==============================] - 11s 308ms/step - loss: 0.0027
Epoch 9/32
36/36 [==============================] - 11s 307ms/step - loss: 0.0025
Epoch 10/32
36/36 [==============================] - 11s 307ms/step - loss: 0.0025
Epoch 11/32
36/36 [==============================] - 12s 342ms/step - loss: 0.0024
Epoch 12/32
36/36 [==============================] - 11s 306ms/step - loss: 0.0023
Epoch 13/32
3